In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from depsurf import FileLogger, check_diff, LinuxImage, GenericCause, OUTPUT_PATH, BuildVersion


def diff_img(img1: LinuxImage, img2: LinuxImage, result_path):
    results = {}
    for key, getter in [
        ("function", lambda img: img.btf.funcs),
        ("struct", lambda img: img.btf.structs),
        # ("union", lambda img: img.btf.unions),
        # ("enum", lambda img: img.btf.enums),
        ("tp_event", lambda img: img.tracepoints.events),
        ("tp_func", lambda img: img.tracepoints.funcs),
        ("lsm", lambda img: img.lsm_hooks),
    ]:
        dict1 = getter(img1)
        dict2 = getter(img2)

        with FileLogger(result_path / f"{key}.log"):
            diff_result = check_diff(dict1, dict2)
        if key in ("tp_event", "tp_func"):
            key = "tracepoint"
        results[(key, "Old")] = len(dict1)
        # results[(key, "New")] = len(dict2)
        results[(key, GenericCause.ADD)] = len(diff_result.added)
        results[(key, GenericCause.REMOVE)] = len(diff_result.removed)
        # results[(key, "Common")] = len(diff_result.common)
        # results[(key, "Changed")] = len(diff_result.changed)
        for k, v in diff_result.reasons.items():
            results[(key, k)] = v

    df = pd.DataFrame(
        results.values(),
        index=pd.MultiIndex.from_tuples(results.keys()),
        columns=["Count"],
    )
    df.to_string(result_path / "summary.txt")
    return df

def diff_img_pairs(group: str, pairs: list):
    results = {}
    print(f"Diffing {group}")
    group_path = OUTPUT_PATH / group
    for v1, v2 in pairs:
        path = group_path / f"{v1}_{v2}"
        print(f"Comparing {v1} and {v2} to {path}")
        df = diff_img(v1.img, v2.img, path)
        results[(group, (v1.name, v2.name))] = df
    df = pd.concat(results, axis=1)
    df.columns = df.columns.droplevel(-1)
    df.to_string(group_path / "summary.txt")
    return df


# diff_img_pairs("test", [(BuildVersion.from_str("5.4.0-26-generic-amd64"), BuildVersion.from_str("5.8.0-25-generic-amd64"))])

In [2]:
from itertools import pairwise

from depsurf import BuildVersion

results = []
for group, versions in [
    ("lts", BuildVersion.filter(lts=True)),
    ("all", BuildVersion.filter()),
    ("rev", BuildVersion.filter(revision="all", version="5.4.0")),
]:
    results.append(diff_img_pairs(group, pairwise(versions)))

[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/4.4.0-21-generic-amd64.pkl


Diffing lts
Comparing 4.4.0-21-generic-amd64 and 4.15.0-20-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/lts/4.4.0-21-generic-amd64_4.15.0-20-generic-amd64


[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/4.15.0-20-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.4.0-21-generic-amd64.pkl
[tracepoint.py:91 ] WARNING: Could not find function trace_event_raw_event_xen_mmu_flush_tlb_all
[   symtab.py:37 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.15.0-20-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-26-generic-amd64.pkl


Comparing 4.15.0-20-generic-amd64 and 5.4.0-26-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/lts/4.15.0-20-generic-amd64_5.4.0-26-generic-amd64


[   symtab.py:37 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.15.0-25-generic-amd64.pkl


...

[   symtab.py:37 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-100-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-125-generic-amd64.pkl


Comparing 5.4.0-100-generic-amd64 and 5.4.0-125-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/rev/5.4.0-100-generic-amd64_5.4.0-125-generic-amd64


[   symtab.py:37 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-125-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-175-generic-amd64.pkl


Comparing 5.4.0-125-generic-amd64 and 5.4.0-175-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/rev/5.4.0-125-generic-amd64_5.4.0-175-generic-amd64


[   symtab.py:37 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-175-generic-amd64.pkl


In [23]:
df = pd.concat(results, axis=1)
df = df.loc[(df != 0).any(axis=1)]
df.to_pickle(OUTPUT_PATH / "diff.pkl")

In [4]:
import pandas as pd
from depsurf import setup_pandas, OUTPUT_PATH

setup_pandas()

pd.set_option("display.expand_frame_repr", True)

pd.read_pickle(OUTPUT_PATH / "diff.pkl")

lts                                                                                                                                                                                                    all                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               \
                               (4.4.0-21-generic-amd64, 4.15.0-20-generic-amd64) (4.15.0-20-generic-amd64, 5.4.0-26-generic-amd64) (5.4.0-26-generic-amd64, 5.15.0-25-generic-amd64) (5.15.0-25-generic-amd64, 6.8.0-22-generic-amd64) (4.4.0-21-generic-amd64, 4.8.0-22-generic-amd64) (4.8.0-22-generic-amd64, 4.10.0-19-generic-amd64) (4.10.0-19-generic-amd64, 4.13.0-16-generic-amd64) (4.13.0-16-generic-amd64, 4.15.0-20-generic-amd64) (4.15.0-20-generic-amd64, 4.18.0-10-generic-amd64) (4.18.0-10-generic-amd64, 5.0.0-13-generic-amd64) (5.0.0-13-generic-amd64, 5.3.0-18-generic-amd64) (5.3.0-18-generic-amd64, 5.4.0-26-generic-amd64) (5.4.0-26-generic-amd64, 5.8.0-25-generic-amd64) (5.8.0-25-generic-amd64, 5.11.0-16-generic-amd64) (5.11.0-16-generic-amd64, 5.13.0-19-generic-amd64) (5.13.0-19-generic-amd64, 5.15.0-25-generic-amd64) (5.15.0-25-generic-amd64, 5.19.0-21-generic-amd64) (5.19.0-21-generic-amd64, 6.2.0-20-generic-amd64) (6.2.0-20-generic-amd64, 6.5.0-9-generic-amd64)   
function   Old                                                             36309                                             42299                                             48018                                             53956                                            36309                                             38211                                              39148                                              40741                                              42299                                             44488                                            45277                                            46992                                            48018                                             51936                                              54302                                              53193                                              53956                                             55843                                           56978   
           Added                                                            8600                                              8929                                             10771                                             12139                                             2849                                              1574                                               2662                                               2118                                               3635                                              1980                                             2516                                             1382                                             6785                                              3518                                               1469                                               1753                                               3525                                              2622                                            3196   
           Removed                                                          2610                                              3210           

In [4]:
from depsurf import BuildVersion

base_version = BuildVersion.from_str("5.4.0-26-generic-amd64")
arch_versions = [
    v
    for v in BuildVersion.filter(version="5.4.0", arch=None)
    if v.arch != "amd64"
]
flavor_versions = [
    v
    for v in BuildVersion.filter(version="5.4.0", flavor=None)
    if v.flavor != "generic"
]

In [5]:
results_arch = diff_img_pairs("arch", [(base_version, v) for v in arch_versions])
results_flavor = diff_img_pairs("flavor", [(base_version, v) for v in flavor_versions])

[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-26-generic-amd64.pkl


Diffing arch
Comparing 5.4.0-26-generic-amd64 and 5.4.0-26-generic-arm64 to /Users/szhong/Downloads/bpf-study/btf/output/arch/5.4.0-26-generic-amd64_5.4.0-26-generic-arm64


[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-26-generic-arm64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-arm64.pkl
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bef0 -> 0
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bef8 -> 0
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bf00 -> 0
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bf08 -> 0
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bf10 -> 0
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bf18 -> 0
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bf20 -> 0
[tracepoint.py:19 ] WARNING: Invalid event pointer: ffff80001160bf28 -> 0
[tracepoint.py:19 ] WARNING: Invalid event 

Comparing 5.4.0-26-generic-amd64 and 5.4.0-26-generic-armhf to /Users/szhong/Downloads/bpf-study/btf/output/arch/5.4.0-26-generic-amd64_5.4.0-26-generic-armhf


[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-armhf.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-26-generic-ppc64el.pkl


...

[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-1009-gcp-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-1009-oracle-amd64.pkl


Comparing 5.4.0-26-generic-amd64 and 5.4.0-1009-oracle-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/flavor/5.4.0-26-generic-amd64_5.4.0-1009-oracle-amd64


[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-1009-oracle-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-1010-azure-amd64.pkl


Comparing 5.4.0-26-generic-amd64 and 5.4.0-1010-azure-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/flavor/5.4.0-26-generic-amd64_5.4.0-1010-azure-amd64


[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-1010-azure-amd64.pkl


In [10]:
import pandas as pd

df = pd.concat([results_arch, results_flavor], axis=1)
df = df.loc[(df != 0).any(axis=1)]
df.to_pickle(OUTPUT_PATH / "config.pkl")
df

arch  \
                               (5.4.0-26-generic-amd64, 5.4.0-26-generic-arm64)   
function   Old                                                            48018   
           Added                                                           9216   
           Removed                                                         7872   
           Param added                                                       63   
           Param removed                                                     63   
           Param type changed                                                57   
           Param reordered                                                    3   
           Return type changed                                                9   
struct     Old                                                             8422   
           Added                                                           1659   
           Removed                                                         1006   
           Field added                                                       33   
           Field removed                                                     39   
           Field type changed                                                37   
tracepoint Old                                                              752   
           Added                                                              0   
           Removed                                                          752   
lsm        Old                                                              189   

                                                                                 \
                               (5.4.0-26-generic-amd64, 5.4.0-26-generic-armhf)   
function   Old                                                            48018   
           Added                                                          12602   
           Removed                                                        11775   
           Param added                                                       50   
           Param removed                                                     50   
           Param type changed                                                55   
           Param reordered                                                    8   
           Return type changed                                               10   
struct     Old                                                             8422   
           Added                                                           1980   
           Removed                                                         1851   
           Field added                                                       42   
           Field removed                                                     66   
           Field type changed                                                89   
tracepoint Old                                                              752   
           Added                                                             70   
           Removed                                                          132   
lsm        Old                                                              189   

                                                                                   \
                               (5.4.0-26-generic-amd64, 5.4.0-26-generic-ppc64el)   
function   Old                                                              48018   
           Added                                                             5444   
           Removed                                                          10636   
           Param added                                                         79   
           Param removed                                                       80   
           Param type changed                                                  61   
           Param reordered                                               